In [0]:
from pyspark.sql.functions import hour, mean, desc, to_date, col , year, split,month, dayofmonth
from pyspark.sql.functions import sum as _sum
from pyspark.sql.functions import *


simpleData = (('i-101','85123A','ABC',150,6,'2021-12-01 08:16:00','c-1001'),


 ('i-102','85124A','XYZ',110,6,'2021-12-01 09:12:00','c-1002'), 


 ('i-103','85125A','MNO',100,4,'2021-12-01 10:00:00','c-1003'), 


 ('i-104','85126A','VWA',102,5,'2021-12-01 10:31:00','c-1004'), 


 ('i-105','85127A','AAS',100,7,'2021-12-01 10:45:00','c-1005'),


 ('i-106','85128A','FAS',130,3,'2021-12-01 11:06:00','c-1006'),


 ('i-107','85129A','AFA',175,6,'2021-12-01 11:15:00','c-1007'),


 ('i-108','85130A','GAG',150,8,'2021-12-01 11:46:00','c-1008'),


 ('i-109','85131A','AGG',180,8,'2021-12-01 12:56:00','c-1009'),


 ('i-110','85132A','KKK',200,1,'2021-12-01 14:36:00','c-1010'))


columns= ['invoice no', 'product code', 'descr','unit price', 'quantity', 'invoice date', 'customer id'] 
df = spark.createDataFrame(data = simpleData, schema = columns)
df.show()

# b . top 3 customer
df.select("customer id").orderBy(desc(col("quantity")*col("unit price"))).show(3)



+----------+------------+-----+----------+--------+-------------------+-----------+
|invoice no|product code|descr|unit price|quantity|       invoice date|customer id|
+----------+------------+-----+----------+--------+-------------------+-----------+
|     i-101|      85123A|  ABC|       150|       6|2021-12-01 08:16:00|     c-1001|
|     i-102|      85124A|  XYZ|       110|       6|2021-12-01 09:12:00|     c-1002|
|     i-103|      85125A|  MNO|       100|       4|2021-12-01 10:00:00|     c-1003|
|     i-104|      85126A|  VWA|       102|       5|2021-12-01 10:31:00|     c-1004|
|     i-105|      85127A|  AAS|       100|       7|2021-12-01 10:45:00|     c-1005|
|     i-106|      85128A|  FAS|       130|       3|2021-12-01 11:06:00|     c-1006|
|     i-107|      85129A|  AFA|       175|       6|2021-12-01 11:15:00|     c-1007|
|     i-108|      85130A|  GAG|       150|       8|2021-12-01 11:46:00|     c-1008|
|     i-109|      85131A|  AGG|       180|       8|2021-12-01 12:56:00|     

In [0]:
df=df.withColumn("sales", col("unit price")*col("quantity"))
df.show()

+----------+------------+-----+----------+--------+-------------------+-----------+-----+
|invoice no|product code|descr|unit price|quantity|       invoice date|customer id|sales|
+----------+------------+-----+----------+--------+-------------------+-----------+-----+
|     i-101|      85123A|  ABC|       150|       6|2021-12-01 08:16:00|     c-1001|  900|
|     i-102|      85124A|  XYZ|       110|       6|2021-12-01 09:12:00|     c-1002|  660|
|     i-103|      85125A|  MNO|       100|       4|2021-12-01 10:00:00|     c-1003|  400|
|     i-104|      85126A|  VWA|       102|       5|2021-12-01 10:31:00|     c-1004|  510|
|     i-105|      85127A|  AAS|       100|       7|2021-12-01 10:45:00|     c-1005|  700|
|     i-106|      85128A|  FAS|       130|       3|2021-12-01 11:06:00|     c-1006|  390|
|     i-107|      85129A|  AFA|       175|       6|2021-12-01 11:15:00|     c-1007| 1050|
|     i-108|      85130A|  GAG|       150|       8|2021-12-01 11:46:00|     c-1008| 1200|
|     i-10

In [0]:
# a . Purchase by customers per hour/Total purchase by hour. 

from pyspark.sql.functions import *
df.select(col("invoice date"), hour(col("invoice date")).alias("hour")).show()

+-------------------+----+
|       invoice date|hour|
+-------------------+----+
|2021-12-01 08:16:00|   8|
|2021-12-01 09:12:00|   9|
|2021-12-01 10:00:00|  10|
|2021-12-01 10:31:00|  10|
|2021-12-01 10:45:00|  10|
|2021-12-01 11:06:00|  11|
|2021-12-01 11:15:00|  11|
|2021-12-01 11:46:00|  11|
|2021-12-01 12:56:00|  12|
|2021-12-01 14:36:00|  14|
+-------------------+----+



In [0]:
# c. Total sales by day
df.groupBy(to_date("invoice date")).agg(sum(col("quantity")* col("unit price"))).alias("total sales").show()


+---------------------+----------------------------+
|to_date(invoice date)|sum((quantity * unit price))|
+---------------------+----------------------------+
|           2021-12-01|                        7450|
+---------------------+----------------------------+



In [0]:
# d.Total sales by year
df.groupBy(year("invoice date")).agg(sum(col("quantity")* col("unit price"))).alias("Total sales").show()

+------------------+----------------------------+
|year(invoice date)|sum((quantity * unit price))|
+------------------+----------------------------+
|              2021|                        7450|
+------------------+----------------------------+



In [0]:
#e. Total sales by quarter 

df.groupBy(quarter("invoice date")) .agg(sum(col("quantity")*  col("unit price"))).alias("total sales").show()


+---------------------+----------------------------+
|quarter(invoice date)|sum((quantity * unit price))|
+---------------------+----------------------------+
|                    4|                        7450|
+---------------------+----------------------------+



In [0]:
# g . sort based on sales
df.orderBy(df.sales.asc()).show()

+----------+------------+-----+----------+--------+-------------------+-----------+-----+
|invoice no|product code|descr|unit price|quantity|       invoice date|customer id|sales|
+----------+------------+-----+----------+--------+-------------------+-----------+-----+
|     i-110|      85132A|  KKK|       200|       1|2021-12-01 14:36:00|     c-1010|  200|
|     i-106|      85128A|  FAS|       130|       3|2021-12-01 11:06:00|     c-1006|  390|
|     i-103|      85125A|  MNO|       100|       4|2021-12-01 10:00:00|     c-1003|  400|
|     i-104|      85126A|  VWA|       102|       5|2021-12-01 10:31:00|     c-1004|  510|
|     i-102|      85124A|  XYZ|       110|       6|2021-12-01 09:12:00|     c-1002|  660|
|     i-105|      85127A|  AAS|       100|       7|2021-12-01 10:45:00|     c-1005|  700|
|     i-101|      85123A|  ABC|       150|       6|2021-12-01 08:16:00|     c-1001|  900|
|     i-107|      85129A|  AFA|       175|       6|2021-12-01 11:15:00|     c-1007| 1050|
|     i-10

In [0]:
# 2nd question

#Create a df for the following. 

#1.                   Add and subtract 5 days from today. 

#2.                   Display startdate and enddate of the current quarter. 

#3.                   Display startdate and enddate of the current month 

In [0]:
df1=spark.createDataFrame([["2022-04-08"]],["currentDate"])

In [0]:
df1.show()

+-----------+
|currentDate|
+-----------+
| 2022-04-08|
+-----------+



In [0]:
from datetime import date
c_date = date.today()
print(c_date)

2022-04-12


In [0]:
#1st question

from pyspark.sql.functions import date_add, date_sub
df1.withColumn("add5days",date_add(df1.currentDate,5)).show()
df1.withColumn("sub5days",date_sub(df1.currentDate,5)).show()

+-----------+----------+
|currentDate|  add5days|
+-----------+----------+
| 2022-04-08|2022-04-13|
+-----------+----------+

+-----------+----------+
|currentDate|  sub5days|
+-----------+----------+
| 2022-04-08|2022-04-03|
+-----------+----------+



In [0]:
from pyspark.sql.functions import trunc,last_day
df1.withColumn("startdate",trunc(df1.currentDate,"month")).show()

+-----------+----------+
|currentDate| startdate|
+-----------+----------+
| 2022-04-08|2022-04-01|
+-----------+----------+



In [0]:
df1.withColumn("enddate",last_day(df1.currentDate)).show()

+-----------+----------+
|currentDate|   enddate|
+-----------+----------+
| 2022-04-08|2022-04-30|
+-----------+----------+



In [0]:
df1.withColumn("startdate",trunc(df1.currentDate,"quarter")).show()

+-----------+----------+
|currentDate| startdate|
+-----------+----------+
| 2022-04-08|2022-04-01|
+-----------+----------+



In [0]:
df1.withColumn("enddate", last_day(date_add(trunc(df1.currentDate,"quarter"),70))).show()

+-----------+----------+
|currentDate|   enddate|
+-----------+----------+
| 2022-04-08|2022-06-30|
+-----------+----------+

